##RAG for Amharic legal data by Yonas Awoke Yitay

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain
print("[INFO] Running in Google Colab, installing requirements.")
!pip install PyMuPDF # for reading PDFs with Python
!pip install tqdm # for progress bars
# !pip install sentence-transformers # for embedding models
# !pip install accelerate # for quantization model loading
# !pip install bitsandbytes # for quantizing models (less storage space)
# !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference
# !pip install sentence-transformers # for embbeding a sentence in to numbers
!pip install langchain langchain_community sentence-transformers torchvision PyMuPDF

[INFO] Running in Google Colab, installing requirements.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 736.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu

In [ ]:
from langchain_core.runnables import Runnable
import google.generativeai as genai

genai.configure(api_key="apii key")
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

class GeminiLLM(Runnable):
    def __init__(self, temperature=0.7):  # Add temperature to init
        self.temperature = temperature

    def invoke(self, input, config=None):
        # input is usually a dict with "messages" or a formatted string prompt
        if isinstance(input, dict) and "messages" in input:
            prompt_str = "\n".join(m.content for m in input["messages"])
        else:
            prompt_str = str(input)

        response = gemini_model.generate_content(
            prompt_str,
            generation_config={"temperature": self.temperature}
        )
        return response.text

# Example usage with temperature
llm = GeminiLLM(temperature=0.3)


In [ ]:
import os
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] =

`(3) API Keys`

In [ ]:
# os.environ['OPENAI_API_KEY'] = <your-api-key>

In [ ]:
import bs4
from langchain import hub

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def rag_pipeline(
    embedding_model,
    documents=None,
    embedding_model_name="default_model",
    prompt=hub.pull("rlm/rag-prompt"),
    k=2,
    persist_base_dir="./drive/MyDrive/rag/legal/vectorstores"

):
    if not os.path.exists(persist_base_dir):

       os.makedirs(persist_base_dir)
    # Set a unique directory for each embedding model
    persist_directory = os.path.join(persist_base_dir, embedding_model_name.replace("/", "_"))

    # Check if vectorstore exists
    if os.path.exists(persist_directory) and documents is None:
        # Load existing vectorstore
        vectorstore = Chroma(
            persist_directory=persist_directory,
            embedding_function=embedding_model
        )
        print(f"[INFO] Loaded existing vectorstore from: {persist_directory}")
    else:
        if documents is None:
            raise ValueError("You must provide documents if no persisted vectorstore is found.")
        # Create and store vectorstore
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=embedding_model,
            persist_directory=persist_directory
        )
        vectorstore.persist()
        print(f"[INFO] Stored new vectorstore at: {persist_directory}")

    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    return (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    ), retriever


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number +1,  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split("፡፡")),
                                "page_token_count":  num_tokens_from_string(text),  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": text})
    return pages_and_texts



In [ ]:
import fitz
from tqdm.auto import tqdm
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number +1,  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split("፡፡")),
                                "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": text})
    return pages_and_texts



In [ ]:
pdf_path="./drive/MyDrive/constitution_amh.pdf"
amh_pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
amh_pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 1660,
  'page_word_count': 307,
  'page_sentence_count_raw': 11,
  'page_token_count': 415.0,
  'text': 'መ ግ ቢ ያ  እኛ የኢትዮጵያ ብሔሮች፣ ብሔረሰቦች፣ ሕዝቦች፡በሀገራችን ኢትዮጵያ ውስጥ ዘላቂ ሰላም፣ ዋስትና ያለው ዴሞክራሲ እንዲሰፍን፣ኢኮኖሚያዊና  ማኅበራዊ እድገታችን እንዲፋጠን፣ የራሳችንን ዕድል በራሳችን የመወሰን መብታችንን ተጠቅመን፣ በነጻ ፍላጐታችን፣ በሕግ የበላይነት እና በራሳችን  ፈቃድ ላይ የተመሰረተ አንድ የፖለቲካ ማኅበረሰብ በጋራ ለመገንባት ቆርጠን በመነሳት፤ይህን ዓላማ ከግብ ለማድረስ፣ የግለሰብና የብሔር/ብሔረሰብ  መሰረታዊ መብቶች መከበራቸው፣ የጾታ እኩልነት መረጋገጡ፣ ባሕሎችና ሃይማኖቶች ካለአንዳች ልዩነት እንዲራመዱ የማድረጉ አስፈላጊነት ጽኑ  እምነታችን በመሆኑ፤ኢትዮጵያ ሀገራችን የየራሳችን አኩሪ ባሕል ያለን፣ የየራሳችን መልክዓ ምድር አሰፋፈር የነበረንና ያለን፣ ብሔር ብሔረሰቦችና  ሕዝቦች በተለያዩ መስኮችና የግንኙነት ደረጃዎችተሳስረንአብረን የኖርንባትና የምንኖርባት ሀገር በመሆንዋ፤ ያፈራነው የጋራ ጥቅምና አመለካከት  አለን ብለን ስለምናምን፤መጪው የጋራ ዕድላችን መመስረት ያለበት ከታሪካችን የወረስነውን የተዛባ ግንኙነት በማረምና የጋራ ጥቅማችንን በማሳደግ  ላይ መሆኑን በመቀበል፤ ጥቅማችንን፣ መብታችንና ነጻነታችንን በጋራ እና በተደጋጋፊነት ለማሳደግ አንድ የኢኮኖሚ ማኅበረሰብ የመገንባቱን  አስፈላጊነት በማመን፤ በትግላችንና በከፈልነው መስዋዕትነት የተገኘውን ዴሞክራሲና ሰላም ዘላቂነቱንለማረጋገጥ፤ይህ ሕገ መንግሥት ከዚህ በላይ  ለገለጽናቸው ዓላማዎችና እምነቶች ማሰሪያ እንዲሆነንእንዲወክሉን መርጠን በ

In [ ]:
import random
random.sample(amh_pages_and_texts, k=3)

[{'page_number': 27,
  'page_char_count': 1032,
  'page_word_count': 207,
  'page_sentence_count_raw': 14,
  'page_token_count': 258.0,
  'text': '11. ስለ ሀገሪቱ ሁኔታ፣ በመንግሥት ስለተከናወኑ ተግባራትና ስለወደፊት እቅዶች ለሕዝብ ተወካዮች ምክር ቤት በየወቅቱ ሪፖረት ያቀርባል፡፡  12. በዚህ ሕገ መንግሥትና በሌሎች ሕጐች የተሰጡትን ሌሎች ተግባሮች ያከናውናል፡፡  13. ሕገ መንግሥቱን ያከብራል፤ የስከበራል፡፡  አንቀጽ 75 ስለ ምክትል ጠቅላይ ሚኒስትር  1. ምክትል ጠቅላይሚኒስትሩ፣  ሀ/ በጠቅላይ ሚኒስትሩ ተለይተው የሚሰጡትን ተግባሮች ያከናውናል፤  ለ/ ጠቅላይ ሚኒስትሩ በማይኖርበት ጊዜ ተክቶት ይሰራል፡፡  2. ምክትል ጠቅላይ ሚኒስትሩ ተጠሪነቱ ለጠቅላይ ሚኒስትሩ ይሆናል፡፡  አንቀጽ 76 የሚኒስትሮች ምክር ቤት  1. የሚኒስትሮች ምክር ቤት፤ ጠቅላይ ሚኒስትር፣ ምክትል ጠቅላይ ሚኒስተር፣  ሚኒስትሮችና በሕግ በሚወሰን መሰረት ሌሎች አባሎች የሚገኙበት ምክር ቤት ነው፡፡  2. የሚኒስትሮች ምክር ቤት ተጠሪነቱ ለጠቅላይ ሚኒሰትሩ ነው፡፡  3. የሚኒስትሮች ምክር ቤት ለሚወስነው ውሳኔ ለሕዝብ ተወካዮች ምክር ቤት ተጠሪ ነው፡፡  አንቀጽ 77 የሚኒስትሮች ምክር ቤት ሥልጣንና ተግባር  1. የሚኒስትሮች ምክር ቤት በሕዝብ ተወካዮች ምክር ቤት የወጡ ሕጐችና የተሰጡ ውሳኔዎች በሥራ መተርጐማቸውን ያረጋግጣል፣ መመሪያዎችን  ይሰጣል፡፡  2. የሚኒስትሮችንና በቀጥታ ለሚኒስትሮች ምክር ቤት ተጠሪ የሆኑ ሌሎች የመንግሥት አካላትን አደረጃጀት ይወስናል፣ ሥራቸውን ያስተባብራል፣  ይመራል፡፡  3. የፌዴራሉን መንግሥት ዓመታዊ በጀት ያዘጋጃል፣ ለሕዝብ ተወካዮች ምክር ቤት ያቀርባል፣

In [ ]:
import pandas as pd

df = pd.DataFrame(amh_pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,1660,307,11,415.00,መ ግ ቢ ያ እኛ የኢትዮጵያ ብሔሮች፣ ብሔረሰቦች፣ ሕዝቦች፡በሀገራችን ኢ...
1,2,969,192,15,242.25,አንቀጽ 5:ስለ ቋንቋ 1. ማናቸውም የኢትዮጵያ ቋንቋዎች በእኩልነት የመ...
2,3,991,199,15,247.75,4. ኢትዮጵያ ያጸደቀቻቸው ዓለም አቀፍ ስምምነቶች የሀገሪቱ ሕግ አካል ና...
3,4,1241,252,13,310.25,አንቀጽ 15 የሕይወት መብት ማንኛውም ሰው በሕይወት የመኖር መብት አለው...
4,5,1666,343,19,416.50,3. የተያዙ ሰዎች በአርባ ስምንት ሰዓታት ውስጥ ፍርድ ቤት የመቅረብ መብ...


In [ ]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,38.000000,38.000000,38.000000,38.000000,38.000000
mean,19.500000,1320.131579,267.736842,14.947368,330.032895
std,11.113055,262.770085,50.733370,3.578822,65.692521
min,1.000000,513.000000,111.000000,4.000000,128.250000
25%,10.250000,1190.500000,245.250000,13.000000,297.625000
50%,19.500000,1363.500000,276.000000,15.000000,340.875000
75%,28.750000,1457.500000,296.750000,17.000000,364.375000
max,38.000000,1826.000000,379.000000,22.000000,456.500000


In [ ]:
for item in tqdm(amh_pages_and_texts):
    # print(item['text'].split("፡፡"))
    item["sentences"] = item['text'].split("፡፡")

    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    # Count the sentences
    item["sentence_split_count"] = len(item["sentences"])


  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(amh_pages_and_texts)
df[['page_sentence_count_raw','sentence_split_count']]

,page_sentence_count_raw,sentence_split_count
0,11,11
1,15,15
2,15,15
3,13,13
4,19,19
5,15,15
6,13,13
7,16,16
8,19,19
9,15,15


In [ ]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,sentence_split_count
count,38.00,38.00,38.00,38.00,38.00,38.00
mean,19.50,1320.13,267.74,14.95,330.03,14.95
std,11.11,262.77,50.73,3.58,65.69,3.58
min,1.00,513.00,111.00,4.00,128.25,4.00
25%,10.25,1190.50,245.25,13.00,297.62,13.00
50%,19.50,1363.50,276.00,15.00,340.88,15.00
75%,28.75,1457.50,296.75,17.00,364.38,17.00
max,38.00,1826.00,379.00,22.00,456.50,22.00


In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 14

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(amh_pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
import random
random.sample(amh_pages_and_texts, k=3)

[{'page_number': 20,
  'page_char_count': 1102,
  'page_word_count': 231,
  'page_sentence_count_raw': 10,
  'page_token_count': 275.5,
  'text': 'ሀ/ ለሕገ መንግሥቱ፤  ለ/ ለሕዝቡ፤ እና  ሐ/ ለሕሊናቸው ብቻ ይሆናሉ፡፡  5. ማንኛውም የምክር ቤቱ አባል በምክር ቤቱ ውስጥ በሚሰጠው ድምጽ ወይም አስተያየት ምክንያት አይከሰስም፡፡ አስተዳደራዊ እርምጃ  አይወሰድበትም፡፡  6. ማንኛውም የምክር ቤቱ አባል ከባድ ወንጀል ሲፈጽም እጅ ከፍንጅ ካለተያዘ በስተቀር ያለ ምክር ቤቱ ፈቃድ አይያዝም፤ በወንጀልም አይከሰስም፡፡  7. ማንኛውም የምክር ቤቱ አባል የመረጠው ሕዝብ አመኔታ ባጣበት ጊዜ በሕግ መሰረት ከምክር ቤት አባልነቱ ይወገዳል፡፡  አንቀጽ 55 የሕዝብ ተወካዮች ምክር ቤት ሥልጣንና ተግባር  1. የሕዝብ ተወካዮች ምክር ቤት በዚህ ሕገ መንግሥት መሰረት ለፌዴራሉ መንግሥት በተሰጠው ሥልጣን ክልል ሕጐችን ያወጣል፡፡  2. በዚህ አንቀጽ ንዑስ አንቀጽ 1 የተመለከተው አጠቃላይ ድንጋጌ እንደተጠበቀ ሆኖ፣ የሕዝብ ተወካዮች ምክር ቤት በሚከተሉት ጉዳዮች ላይ  ዝርዝር ሕግ ያወጣል፤  ሀ/ የመሬትና የተፈጥሮ ሀብት፤ እንዲሁም ድንበር ተሻጋሪ ወይም ከአንድ ክልል በላይ የሚያስተሳስሩ ወንዞችና ሐይቆች አጠቃቀምን በተመለከተ፤  ለ/ በክልሎች መካከል የሚኖረውን የንግድ ልውውጥ፤ እንዲሁም የውጭ ንግድ ግንኙነትን በተመለከተ፤  ሐ/ የአየር፣ የባቡርና የባሕር መጓጓዣ፤ የፖስታና የቴሌኮሙኒኬሽን አገልግሎቶችን እንዲሁም ሁለት ወይም ከሁለት በላይ ክልሎችን የሚያገናኙ  አውራ መንገዶችን በተመለከተ፤  መ/ በዚህ ሕገ መንግሥት የተደነገጉትን የፖለቲካ መብቶች አፈጻጸምን እንዲሁም ምርጫን በተመለከተ፤

In [ ]:
# Create a DataFrame to get stats
df = pd.DataFrame(amh_pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,sentence_split_count,num_chunks
count,38.00,38.00,38.00,38.00,38.00,38.00,38.00
mean,19.50,1320.13,267.74,14.95,330.03,14.95,1.63
std,11.11,262.77,50.73,3.58,65.69,3.58,0.49
min,1.00,513.00,111.00,4.00,128.25,4.00,1.00
25%,10.25,1190.50,245.25,13.00,297.62,13.00,1.00
50%,19.50,1363.50,276.00,15.00,340.88,15.00,2.00
75%,28.75,1457.50,296.75,17.00,364.38,17.00,2.00
max,38.00,1826.00,379.00,22.00,456.50,22.00,2.00


In [ ]:
import re

# Split each chunk into its own item
amh_pages_and_chunks = []
for item in tqdm(amh_pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        # joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        amh_pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(amh_pages_and_chunks)

  0%|          | 0/38 [00:00<?, ?it/s]

62

In [ ]:
random.sample(amh_pages_and_chunks, k=3)

[{'page_number': 34,
  'sentence_chunk': 'ለ/ የተፈጥሮ አደጋ ሲያጋጥም ወይም የሕዝብን ደህንነት አደጋ ላይ የሚጥል በሽታ ሲከሰት የክልል መስተዳድሮች በክልላቸው የአስቸኳይ ጊዜ አዋጅ ሊያውጁ ይችላሉዝርዝሩ ክልሎች ይህን ሕገ መንግሥት መሰረት በማድረግ በሚያወጧቸው ሕገመንግሥቶች ይወሰናል 2. በዚህ አንቀጽ ንዑስ አንቀጽ 1 /ሀ/ መሰረት የሚታወጅ የአስቸኳይ ጊዜ አዋጅ፣ ሀ/ የሕዝብ ተወካዮች ምክር ቤት በሥራ ላይ ባለ ጊዜ የታወጀ ከሆነ በታወጀ በአርባ ስምንት ሰዓታት ውስጥ ለሕዝብ ተወካዮች ምክር ቤት መቅረብ አለበት አዋጁ በሕዝብ ተወካዮች ምክር ቤት ሁለት ሦስተኛው ድምጽ ተቀባይነት ካላገኘ ወዲያውኑ ይሻራል ለ/ ከላይ በንዑስ አንቀጽ /ሀ/ ሥር የተጠቀሰው እንደተጠበቀ ሆኖ፣ የሕዝብ ተወካዮች ምክር ቤት በሥራ ላይ ባልሆነበት ወቅት የሚታወጅ የአስቸኳይ ጊዜ አዋጅ ለሕዝብ ተወካዮች ምክር ቤት መቅረብ ያለበት አዋጁ በታወጀ በአሥራ አምሥት ቀናት ውስጥ ነው 3. በሚኒስትሮች ምክር ቤት የተደነገገው የአስቸኳይ ጊዜ አዋጅ በምክር ቤቱ ተቀባይነት ካገኘ በኝላ ሊቆይ የሚችለው እስከ ስድስት ወራት ነው የሕዝብ ተወካዮች ምክር ቤት በሁለት ሦስተኛ ድምጽ አንድን የአስቸኳይ ጊዜ አዋጅ በየአራት ወሩ በተደጋጋሚ እንዲታደስ ሊያደርግ ይችላል 4. ሀ/ የአስቸኳይ ጊዜ አዋጅ በሚታወጅበት ጊዜ የሚኒስትሮች ምክር ቤት በሚያወጣቸው ደንቦች መሰረት የሀገርን ሰላምና ሕልውና፣ የሕዝብን ደህንነት፣ሕግና ሥርዓትን የማስከበር ሥልጣን ይኖረዋል ለ/ የሚኒስትሮች ምክር ቤት ሥልጣን በሕገ መንግሥቱ የተቀመጡትን መሰረታዊ የፖለቲካና የዴሞክራሲ መብቶችን፣ የአስቸኳይ ጊዜ አዋጁን ለማወጅ ምክንያት የሆነውን ጉዳይ ለማስወገድ ተፈላጊ ሆኖ በተገኘውደረጃ፣ እስከ ማገድ ሊደርስ የ

In [ ]:
# Get stats about our chunks
df = pd.DataFrame(amh_pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,62.00,62.00,62.00,62.00
mean,19.60,777.98,150.45,194.50
std,10.97,541.19,104.03,135.30
min,1.00,0.00,1.00,0.00
25%,10.00,198.75,40.25,49.69
50%,20.50,952.50,180.50,238.12
75%,28.75,1190.25,235.50,297.56
max,38.00,1778.00,355.00,444.50


In [ ]:
# Show random chunks with under 30 tokens in length if they are worth watching
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 10.25 | Text: አንቀጽ 68 በሁለቱም ምክር ቤቶች አባል መሆን የማይቻል ስለመሆኑ
Chunk token count: 24.25 | Text: 3. መንግሥት በዓለም አቀፍ ደረጃ የሚገባቸው ስምምነቶችም ሆኑ የሚያደርጋቸው ግንኙነቶች የኢትዮጵያን የማያቋርጥ እድገት መብት የሚያስከብሩ መሆን አለባቸው
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 


In [ ]:
df[df["chunk_token_count"] <= min_token_length]

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
2,2,,0,1,0.00
4,3,,0,1,0.00
9,6,,0,1,0.00
16,10,,0,1,0.00
22,14,3. መንግሥት በዓለም አቀፍ ደረጃ የሚገባቸው ስምምነቶችም ሆኑ የሚያደርጋ...,97,17,24.25
34,22,,0,1,0.00
36,23,,0,1,0.00
38,24,አንቀጽ 68 በሁለቱም ምክር ቤቶች አባል መሆን የማይቻል ስለመሆኑ,41,9,10.25
40,25,6. በጠቅላይ ሚኒስትሩ አቅራቢነት በሕግ በተወሰነው መሰረት ከፍተኛ የውት...,110,22,27.50
50,31,5. ከጐረቤት ሀገሮችና ከሌሎችም የአፍሪካ ሀገሮች ጋር በየጊዜው እያደገ ...,85,15,21.25


In [ ]:
# Show random chunks with 0 tokens
for row in df[df["chunk_token_count"] == 0].iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 
Chunk token count: 0.0 | Text: 


In [ ]:
amh_pages_and_chunks = df[df["chunk_token_count"] >0].to_dict(orient="records")

In [ ]:
amh_pages_and_chunks

[{'page_number': 1,
  'sentence_chunk': 'መ ግ ቢ ያ እኛ የኢትዮጵያ ብሔሮች፣ ብሔረሰቦች፣ ሕዝቦች፡በሀገራችን ኢትዮጵያ ውስጥ ዘላቂ ሰላም፣ ዋስትና ያለው ዴሞክራሲ እንዲሰፍን፣ኢኮኖሚያዊና ማኅበራዊ እድገታችን እንዲፋጠን፣ የራሳችንን ዕድል በራሳችን የመወሰን መብታችንን ተጠቅመን፣ በነጻ ፍላጐታችን፣ በሕግ የበላይነት እና በራሳችን ፈቃድ ላይ የተመሰረተ አንድ የፖለቲካ ማኅበረሰብ በጋራ ለመገንባት ቆርጠን በመነሳት፤ይህን ዓላማ ከግብ ለማድረስ፣ የግለሰብና የብሔር/ብሔረሰብ መሰረታዊ መብቶች መከበራቸው፣ የጾታ እኩልነት መረጋገጡ፣ ባሕሎችና ሃይማኖቶች ካለአንዳች ልዩነት እንዲራመዱ የማድረጉ አስፈላጊነት ጽኑ እምነታችን በመሆኑ፤ኢትዮጵያ ሀገራችን የየራሳችን አኩሪ ባሕል ያለን፣ የየራሳችን መልክዓ ምድር አሰፋፈር የነበረንና ያለን፣ ብሔር ብሔረሰቦችና ሕዝቦች በተለያዩ መስኮችና የግንኙነት ደረጃዎችተሳስረንአብረን የኖርንባትና የምንኖርባት ሀገር በመሆንዋ፤ ያፈራነው የጋራ ጥቅምና አመለካከት አለን ብለን ስለምናምን፤መጪው የጋራ ዕድላችን መመስረት ያለበት ከታሪካችን የወረስነውን የተዛባ ግንኙነት በማረምና የጋራ ጥቅማችንን በማሳደግ ላይ መሆኑን በመቀበል፤ ጥቅማችንን፣ መብታችንና ነጻነታችንን በጋራ እና በተደጋጋፊነት ለማሳደግ አንድ የኢኮኖሚ ማኅበረሰብ የመገንባቱን አስፈላጊነት በማመን፤ በትግላችንና በከፈልነው መስዋዕትነት የተገኘውን ዴሞክራሲና ሰላም ዘላቂነቱንለማረጋገጥ፤ይህ ሕገ መንግሥት ከዚህ በላይ ለገለጽናቸው ዓላማዎችና እምነቶች ማሰሪያ እንዲሆነንእንዲወክሉን መርጠን በላክናቸው ተወካዮቻቸን አማካይነት በሕገ መንግሥት ጉባኤ ዛሬ ኅዳር 29 ቀን 1987 አጽድቀነዋል ምዕራፍ አንድ : ጠቅላላ ድንጋጌዎች አንቀጽ 1: የኢትዮጵያ መንግሥት ስያሜ ይህ ሕገ 

In [ ]:
from langchain.docstore.document import Document

# ... (rest of your code) ...

# Convert chunks (strings) to Document objects
documents = [Document(page_content=doc["sentence_chunk"] ,metadata={'source': pdf_path,'page': doc['page_number']}) for doc in amh_pages_and_chunks]

len(documents)

54

In [ ]:
from langchain import PromptTemplate

prompt = PromptTemplate.from_template(
    "ከታች ያለው መረጃን በመጠቀም፣ የተጠየቀውን ጥያቄ መልስ።\n\nማብራሪያ:\n{context}\n\nጥያቄ: {question}\nመልስ:"
)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

multilingual_e5_large = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",

)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
multilingual_e5_large_chain, multilingual_e5_large_retriever=rag_pipeline(embedding_model=multilingual_e5_large,prompt=prompt,embedding_model_name="multilingual_e5_large",k=5)

<ipython-input-12-2b89a91e7fb3>:35: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


[INFO] Loaded existing vectorstore from: ./drive/MyDrive/rag/legal/vectorstores/multilingual_e5_large


In [ ]:
amh_questions=["ሰባዊ መብት  ምንድነ ነው?","አስቸኳይ ጊዜ አዋጅ?"," የሰባዊ መብቶች እንድጠቅሽልኝ እፈልጋለሁ?","የዲሞክራሲ መብቶች እንድጠቅሽልኝ እፈልጋለሁ?","ስለክልል ከፍተኛ ፍርድ ቤቶች የዳኝነት ስልጣን እና ልዩ ፍርድ ቤቶች አወቃቀር"]

In [ ]:
import tqdm as tqdm_progress # Import tqdm with a different name

def answer_question( rag_chain, retiver ,embedding_model,questions=amh_questions ):

  print(f"[INFO] Answering {len(questions)} questions using model {embedding_model}")
  # Use the renamed tqdm_progress variable
  for question in tqdm_progress.tqdm(questions):
    print("Question",question)
    print(f"Answer: {rag_chain.invoke(question)}\n")

In [ ]:
answer_question(multilingual_e5_large_chain,multilingual_e5_large_retriever,"amh_multilingual_e5_large_chain")

[INFO] Answering 5 questions using model amh_multilingual_e5_large_chain


  0%|          | 0/5 [00:00<?, ?it/s]

Question ሰባዊ መብት  ምንድነ ነው?


 20%|██        | 1/5 [00:08<00:34,  8.67s/it]

Answer: ሰባዊ መብቶች ከሰው ልጅ ተፈጥሮ የሚመነጩ፣ የማይጣሱና የማይገፈፉ ናቸው። (አንቀጽ 10(1))


Question አስቸኳይ ጊዜ አዋጅ?


...

Answer: እርግጥ ነው! ከላይ ካለው ጽሑፍ የተወሰኑ የሰብዓዊ መብቶች እነሆ:

*   ሰብዓዊ ክብራቸውን በሚጠብቁ ሁኔታዎች የመያዝ መብት
*   ከቅርብ ዘመዶቻቸው፣ ከጓደኞቻቸው፣ ከሃይማኖት አማካሪዎቻቸው፣ ከሐኪሞቻቸው እና ከሕግ አማካሪዎቸው ጋር ለመገናኘትና እንዲጐበቿቸውም ዕድል የማግኘት መብት
*   የሃይማኖት፣ የእምነትና የአመለካከት ነጻነት
*   የአመለካከት እና ሐሳብን በነጻ የመያዝና የመግለጽ መብት
*   የግል ሕይወት የመከበርና የመጠበቅ መብት
*   የክብርና የመልካም ስም መብት
*   የእኩልነት መብት
*   ሴቶች ከወንዶች ጋር እኩል መብት አላቸው

ከላይ የተዘረዘሩት ጥቂቶቹ ናቸው። ማንኛውም ጥያቄ ካለዎት, ለመጠየቅ አያመንቱ.

Question የዲሞክራሲ መብቶች እንድጠቅሽልኝ እፈልጋለሁ?


 80%|████████  | 4/5 [00:18<00:04,  4.03s/it]

Answer: እሺ፣ ከላይ ካለው መረጃ በመነሳት የዲሞክራሲ መብቶችን እንደሚከተለው ዘርዝሬልሃለሁ፡

*   **ሐሳብን የመግለጽ ነጻነት** (አንቀጽ 30/2)፡ ማንኛውም ሰው ያለማንም ጣልቃ ገብነት ሐሳቡን የመግለጽ ነጻነት አለው። ይህም በቃል፣ በጽሑፍ፣ በሕትመት፣ በሥነ ጥበብ ወይም በሌሎች የማሰራጫ ዘዴዎች መረጃንና ሐሳብን የመሰብሰብ፣ የመቀበልና የማሰራጨት ነጻነትን ያካትታል።

*   **የፕሬስና የመገናኛ ብዙኃን ነጻነት** (አንቀጽ 30/3)፡ የፕሬስና የሌሎች መገናኛ ብዙኃን ነጻነት የተረጋገጠ ሲሆን በተለይም የቅድሚያ ምርመራ በማንኛውም መልኩ የተከለከለ መሆኑን እና የሕዝብን ጥቅም የሚመለከት መረጃ የማግኘት ዕድልን ያካትታል።

*   **የመሰብሰብ፣ ሰላማዊ ሰልፍ የማድረግና አቤቱታ የማቅረብ መብት** (አንቀጽ 31/1)፡ ማንኛውም ሰው ከሌሎች ጋር በመሆን መሣሪያ ሳይዝ በሰላም የመሰብሰብ፣ ሰላማዊ ሰልፍ የማድረግና አቤቱታ የማቅረብ መብት አለው።

*   **የመደራጀት መብት** (አንቀጽ 32)፡ ማንኛውም ሰው ለማንኛውም ዓላማ በማኅበር የመደራጀት መብት አለው።

Question ስለክልል ከፍተኛ ፍርድ ቤቶች የዳኝነት ስልጣን እና ልዩ ፍርድ ቤቶች አወቃቀር


100%|██████████| 5/5 [00:22<00:00,  4.54s/it]

Answer: ## ስለክልል ከፍተኛ ፍርድ ቤቶች የዳኝነት ስልጣን እና ልዩ ፍርድ ቤቶች አወቃቀር

**የክልል ከፍተኛ ፍርድ ቤቶች የዳኝነት ስልጣን፡**

*   በክልሉ ከሚኖረው የዳኝነት ሥልጣን በተጨማሪ የፌዴራል የመጀመሪያ ደረጃ ፍርድ ቤት የዳኝነት ሥልጣን ይኖረዋል። (አንቀጽ 78(2))
*   በፌዴራል የመጀመሪያ ደረጃ ፍርድ ቤት የዳኝነት ሥልጣኑ መሰረት በሚሰጠው ውሳኔ ላይ የሚቀርበው ይግባኝ በክልል ጠቅላይ ፍርድ ቤት ይታያል።
*   የክልል ከፍተኛ ፍርድ ቤት ዳኞች በክልሉ የዳኞች አስተዳደር ጉባኤ አቅራቢነት በክልሉ ምክር ቤት ይሾማሉ። (አንቀጽ 81(4))

**ልዩ ፍርድ ቤቶች አወቃቀር፡**

*   የዳኝነት ሥልጣንን ከመደበኛ ፍርድ ቤቶች ወይም በሕግ የመዳኘት ሥልጣን ከተሰጠው ተቋም ውጭ የሚያደርግ፣ በሕግ የተደነገገን የዳኝነት ሥርዓት የማይከተል ልዩ ፍርድ ቤት ወይም ጊዜያዊ ፍርድ ቤት አይቋቋምም። (አንቀጽ 83(4))
*   ይሁን እንጂ ሕገ መንግሥቱ ሃይማኖትና የባሕል ፍርድ ቤቶችን ሊያቋቁሙ ወይም እውቅና ሊሰጡ እንደሚችሉ ይደነግጋል።
*   ይህ ሕግ ከመጽደቁ በፊት በመንግሥት እውቅና አግኝተው ሲሰራባቸው የነበሩ ሃይማኖቶችና የባሕል ፍርድ ቤቶች በዚህ ሕገ መንግሥት መሰረት እውቅና አግኝተው ይደራጃሉ። (አንቀጽ 83(4))

በአጭሩ፣ የክልል ከፍተኛ ፍርድ ቤቶች ሁለቱንም የክልል እና የፌዴራል የመጀመሪያ ደረጃ ፍርድ ቤቶች ስልጣን ሲኖራቸው፣ ልዩ ፍርድ ቤቶች በህግ የተደነገገውን የዳኝነት ስርዓት መከተል አለባቸው፣ ነገር ግን ሃይማኖታዊ እና ባህላዊ ፍርድ ቤቶች ሊፈቀዱ ይችላሉ።



In [ ]:

from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines please no other word is needed just the questions. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
question = "ስለ አስቸኳይ ጊዜ አዋጅ?"
generate_queries.invoke("ስለ አስቸኳይ ጊዜ አዋጅ?")

['ስለ አስቸኳይ ጊዜ አዋጅ ምን መረጃ አለ?',
 'የአስቸኳይ ጊዜ አዋጅ አላማ ምንድን ነው?',
 'የአስቸኳይ ጊዜ አዋጅ የሚያስከትላቸው ውጤቶች ምንድን ናቸው?',
 'በኢትዮጵያ ስለተላለፈው የአስቸኳይ ጊዜ አዋጅ ወቅታዊ መረጃ',
 'የአስቸኳይ ጊዜ አዋጅ እንዴት ይተገበራል?',
 '']

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retrieval_chain = generate_queries | multilingual_e5_large_retriever.map() | get_unique_union

docs = retrieval_chain.invoke({"question":question})
len(docs)

docs
# # Run

# questions = generate_queries.invoke({"question":question})

[Document(metadata={'page': 38, 'source': './drive/MyDrive/constitution_amh.pdf'}, page_content='ሀ/ ሁሉም የክልል ምክር ቤቶች የቀረበውን ማሻሻያ በድምጽ ብልጫ ሲያጸድቁት፣ ለ/ የፌዴራሉ መንግሥት የሕዝብ ተወካዮች ምክር ቤት በሁለት ሦስተኛ ድምጽ የቀረበውን ማሻሻያ ሲያጸድቀው፣ እና ሐ/ የፌዴሬሽኑ ምክር ቤት በሁለት ሦስተኛ ድምጽ ማሻሻያውን ሲያጸድቀው ነው 2. በዚህ አንቀጽ ንዑስ አንቀጽ 1 ከተዘረዘሩት ውጭ ያሉት የሕገ መንግሥቱ ድንጋጌዎች ሊሻሻሉ የሚችሉት በሚከተለው አኳኊን ብቻ ይሆናል፤ ሀ/ የሕዝብ ተወካዮች ምክር ቤትና የፌዴሬሽኑ ምክር ቤት በጋራ ስብሰባ በሁለት ሦስተኛ ድምጽ የቀረበውን ማሻሻያ ሲያጸድቁት፣ እና ለ/ ከፌዴሬሽኑ አባል ክልሎቸ ምክር ቤቶች ውስጥ የሁለት ሦስተኛ ክልሎች ምክር ቤቶች በድምጽ ብልጫ የቀረበውን ማሻሻያ ሲያጸድቁት ነው አንቀጽ 106 የመጨረሻ ሕጋዊ እውቅና ስላለው ቅጂ የዚህ ሕገመንግሥት የአማርኛ ቅጂ የመጨረሻው ሕጋዊ እውቅና ያለው ሰነድ ነው'),
 Document(metadata={'page': 21, 'source': './drive/MyDrive/constitution_amh.pdf'}, page_content='5. የወንጀለኛ መቅጫ ሕግ ያወጣል ይህ እንደተጠበቀ ሁኖ ክልሎች የፌዴራሉ መንግሥት የወንጀለኛ መቅጫ ሕግ በግልጽ ባልተሸፈኑ ጉዳዮች ላይ ሕግ የማውጣት ሥልጣን ይኖራቸዋል 6. አንድ የኢኮኖሚ ማኅበረሰብን ለመፍጠር ሲባል በፌዴራል መንግሥት ሕግ እንዲወጣላቸው የሚያስገድዱ ለመሆናቸው በፌዴሬሽኑ ምክር ቤት የታመነባቸው የፍትሐብሔር ሕጐችን ያወጣል 7. የፌዴራል መንግሥት፤ የሀገርና የሕዝብ መከላከያ፤ የደህንነትና የፖሊስ ኃይል አደረጃጀት ይወስናል በሥራ አፈጻጸም ረገድ የሚታዩ መ

##Genrative chain

In [ ]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question in Amaharic based on this context if you dont know the answer say i don know it:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)



final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'በአስቸኳይ ጊዜ አዋጅ ላይ በተመለከተው መረጃ መሰረት፡\n\n*   የውጭ ወረራ ሲያጋጥም ወይም ሕገ መንግሥታዊ ሥርዓቱን አደጋ ላይ የሚጥል ሁኔታ ሲከሰትና በተለመደው የሕግ ማስከበር ሥርዓት ለመቋቋም የማይቻል ሲሆን፣ ማናቸውም የተፈጥሮ አደጋ ሲያጋጥም ወይም የሕዝብን ጤንነት አደጋ ላይ የሚጥል በሽታ ሲከሰት፣ የፌዴራሉ መንግሥት የሚኒስትሮች ምክር ቤት የአስቸኳይ ጊዜ አዋጅ የመደንገግ ሥልጣን አለው።\n*   የክልል መስተዳድሮችም በተመሳሳይ ሁኔታ በክልላቸው የአስቸኳይ ጊዜ አዋጅ ሊያውጁ ይችላሉ።\n*   የአስቸኳይ ጊዜ አዋጅ ከታወጀ በኋላ ለሕዝብ ተወካዮች ምክር ቤት መቅረብና በሁለት ሦስተኛ ድምጽ መጽደቅ አለበት።\n*   የአስቸኳይ ጊዜ አዋጅ አፈጻጸምን ለመቆጣጠር የአስቸኳይ ጊዜ አዋጅ አፈጻጸም መርማሪ ቦርድ ይቋቋማል።\n\n'

ERROR:tornado.access:500 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 41780.62ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: TypeError: Failed to fetch